In [4]:
import json
import requests

from googlesearch import search

import praw

from multiprocess import Pool
import time

from g4f.client import Client

In [5]:
query = "best res"

cursor_pos = len(query)
url_friendly_query = query.replace(" ", "%20")

url = f"https://www.google.com/complete/search?q={url_friendly_query}%20reddit&cp={cursor_pos}&client=gws-wiz&xssi=t&hl=en&authuser=0&dpr=2"

res = requests.get(url).text[5:]
parsed = json.loads(res)[0]

In [6]:
suggestions = [element[0].replace("reddit", "").replace(" </b>", "</b>") for element in parsed]

In [7]:
suggestions

['best res<b>ume builder</b>',
 'best res<b>ume template</b>',
 'best res<b>ident evil game</b>',
 'best res<b>istance bands</b>',
 'best res<b>taurants nyc</b>',
 'best res<b>taurants chicago</b>',
 'best res<b>ume writing service</b>',
 'best res<b>taurants toronto</b>',
 'best res<b>taurants in rome</b>',
 'best res<b>taurants london</b>']

In [59]:
query = "best budgeting app reddit"

In [60]:
submission_ids = []
for res in search(query, safe=None):
    if "/comments/" in res:
        submission_id = res.split("/")[6]
        submission_ids.append(submission_id)

In [61]:
r = praw.Reddit(
    client_id="2gs3ULbVK-pGh7INla0gWg",
    client_secret="g6S2REUlWw8fBBIgJiaFrqIkZUVICQ",
    user_agent="web:tech.redditrecs:v1.0.0 (by u/matetoes)",
)

In [62]:
submission_ids

['18mhmo1',
 '1ceo86f',
 '17lj2md',
 '1dnf5ma',
 '1bf0wqy',
 '18hs1ph',
 '1grfpey',
 '1ajrhra',
 'rtznth',
 '1f41co8']

In [63]:
submissions = []

def jsonify(comment):
    return {
        "id": comment.id,
        "body": comment.body,
        "score": comment.score,
        "author": str(comment.author),
        "replies": [jsonify(reply) for reply in comment.replies],
    }

def get(submission_id):
    # let's time how long this takes
    start = time.time()
    comments = []
    submission = r.submission(submission_id)
    submission.comments.replace_more(limit=None)
    # get title and body of submission
    title = submission.title
    selftext = submission.selftext
    for comment in submission.comments:
        comments.append(jsonify(comment))
    result = {
        "title": title,
        "selftext": selftext,
        "comments": comments,
    }
    end = time.time()
    print(f"Submission {submission_id} took {end - start} seconds")
    return result

with Pool(len(submission_ids)) as pool:
    submissions = pool.map(get, submission_ids)

Submission 1ajrhra took 0.54341721534729 seconds
Submission 18mhmo1 took 0.5704689025878906 seconds
Submission rtznth took 0.5930249691009521 secondsSubmission 1dnf5ma took 0.5943222045898438 seconds

Submission 1bf0wqy took 0.6290040016174316 seconds
Submission 18hs1ph took 0.6888818740844727 seconds
Submission 1ceo86f took 0.8243181705474854 seconds
Submission 1grfpey took 0.8786208629608154 seconds
Submission 1f41co8 took 1.0432429313659668 seconds
Submission 17lj2md took 1.9688961505889893 seconds


In [64]:
len(json.dumps(submissions[0]["comments"][0]))

# let's find the max
max_len = 0
max_comment = None
for submission in submissions:
    for comment in submission["comments"]:
        if len(json.dumps(comment)) > max_len:
            max_len = len(json.dumps(comment))
            max_comment = comment

max_len

10035

In [65]:
len(submissions[0]["comments"])

# let's sum all the lengths
print(sum([len(submission["comments"]) for submission in submissions]))

222


In [66]:
def to_input(submission, query):
    return {
        "query": query,
        "title": submission["title"],
        "selftext": submission["selftext"],
        "comment_thread": submission["comments"][0],
    }

In [67]:
example_inputs = []

In [68]:
example_inputs.append(to_input(submissions[0], query))

In [69]:
print(f"test: {example_inputs[0]}")

test: {'query': 'best budgeting app reddit', 'title': 'Best budgeting app for a complete newbie?', 'selftext': "42 years old and have a lot of fear around money. That fear keeps me from really tracking my personal finances. Like, I literally check my bank account balance a week before rents due and just hope for the best lol. So far I've always been fine in my adult life and able to pay rent; although there have been some close calls. I'm so mindful in so many other aspects of my life. But just not when it comes to money, budgeting, and tracking. I know the way I do things now is INCREDIBLY irresponsible. \n\n  \nMy goal for 2024 is to lean into that fear around money and to start tracking my personal finance, budgeting, start being more mindful of my spending, and to just all around become more financially responsible.\n\n  \nIs there a budgeting app you'd recommend for a beginner? Also any other tips?\n\n  \nThanks in advance", 'comment_thread': {'id': 'ke4p04e', 'body': 'Another vot

In [70]:
example_outputs = []

In [71]:
example_out1 = example_inputs[0].copy()

In [72]:
example_out1

{'query': 'best budgeting app reddit',
 'title': 'Best budgeting app for a complete newbie?',
 'selftext': "42 years old and have a lot of fear around money. That fear keeps me from really tracking my personal finances. Like, I literally check my bank account balance a week before rents due and just hope for the best lol. So far I've always been fine in my adult life and able to pay rent; although there have been some close calls. I'm so mindful in so many other aspects of my life. But just not when it comes to money, budgeting, and tracking. I know the way I do things now is INCREDIBLY irresponsible. \n\n  \nMy goal for 2024 is to lean into that fear around money and to start tracking my personal finance, budgeting, start being more mindful of my spending, and to just all around become more financially responsible.\n\n  \nIs there a budgeting app you'd recommend for a beginner? Also any other tips?\n\n  \nThanks in advance",
 'comment_thread': {'id': 'ke4p04e',
  'body': 'Another vote

In [ ]:
system_msg = "You are a helpful assistant."
user_msg = """I will give you a search query (e.g., "best headphones reddit"), a Reddit post that resulted from googling that query, and a comment thread from that post in the form of a JSON object. Reply with a JSON array of items mentioned in the comment thread. Each element in the array will have three keys: `name`, which is simply the name of the item; `recommendedBy`, which is a list of comment ids that recommend the item; and `notRecommendedBy`, which is a list of comment ids that don't recommend the item and instead actively dissuade others from using the item. Let's go through some examples:

Input 1: {'query': 'best git mergetool',
 'title': 'What diff / merge tools do you use with Git?',
 'selftext': "I have a Mac and have been using Visual Studio Code as my diff and merge tool and it works pretty well but I'm curious what other people use (even if they are not Mac users).\n\nIs there a specific reason you use the tool that you do?",
 'comment_thread': {'id': 'kbfzc62',
  'body': 'I just use the built in FileMerge tool on Mac, or the built in diff tooling in the Fork git client. \nAs for the reason, it’s because it’s just the first one I used, or the most convenient as I don’t need to install another product.',
  'score': 3,
  'author': 'somore_nick',
  'replies': [{'id': 'kbgbc7o',
    'body': 'Thank you.',
    'score': 1,
    'author': 'CromulentSlacker',
    'replies': []}]}}
Output 1: [
    {
        "name": "FileMerge",
        "recommendedBy": ["kbfzc62"],
        "notRecommendedBy": []
    }
]

Input 2: {'query': 'best baas reddit',
 'title': 'What are your favorite BaaS and Databases alternatives to Firebase?',
 'selftext': "I have been using Firebase for a long time and I'm generally a fan. It makes developing a dedicated backend obsolete because it combines a Database, Authentication, Functions, etc, and the Flutter api is easy to use. Therefore saves me a lot of time. For normal apps, I no longer want to use a dedicated backend.\n\nHowever, the only downside for me is Firestore and its limitations. Imo it just has quite a lot of limitations to how I query and manage data. For example, full-text search is something I need in many apps but not possible in Firestore without using third-party solutions like Algolia, which I have used successfully before but it's just not optimal to use two services where the data between each other needs to be synchronized. Another example, for example, I want to update all documents based on a certain conditions means I need to first query all these documents and then update them.\n\nThese points make me think about using an SQL database as it seems that I have more control over db operations.\n\nIt seems, Firebase is by far the most popular option for a BaaS for Flutter. I know there are other solutions in MongoDB, Supabase, etc. but haven't yet tried them in detail.\n\nWhat are your favorite backend solutions for Flutter development and how do they compare to Firebase? What are the limitations that other BaaS have?",
 'comment_thread': {'id': 'kifgm3d',
  'body': 'Take a look at pocketbase. Uses sqllite for db and very easy to setup.',
  'score': 12,
  'author': 'rip71de',
  'replies': [{'id': 'kih8njp',
    'body': 'PocketBase is great.',
    'score': 2,
    'author': 'strash_one',
    'replies': []}]}}
Output 2: [
    {
        "name": "PocketBase",
        "recommendedBy": ["kifgm3d"],
        "notRecommendedBy": []
    }
]

Input 3: {'query': 'best budgeting app reddit',
 'title': 'Best budgeting app for a complete newbie?',
 'selftext': "42 years old and have a lot of fear around money. That fear keeps me from really tracking my personal finances. Like, I literally check my bank account balance a week before rents due and just hope for the best lol. So far I've always been fine in my adult life and able to pay rent; although there have been some close calls. I'm so mindful in so many other aspects of my life. But just not when it comes to money, budgeting, and tracking. I know the way I do things now is INCREDIBLY irresponsible. \n\n  \nMy goal for 2024 is to lean into that fear around money and to start tracking my personal finance, budgeting, start being more mindful of my spending, and to just all around become more financially responsible.\n\n  \nIs there a budgeting app you'd recommend for a beginner? Also any other tips?\n\n  \nThanks in advance",
 'comment_thread': {'id': 'ke4p04e',
  'body': 'Another vote for YNAB. Since it\'s your first budgeting experience, you don\'t need to unlearn bad habits. I\'ve been a user for years and their methodology changed how I manage my money. \n\nYnab is a zero based digital envelope system. It asks you to take all the money you have, dump it on a table and then put it into envelopes (app calls them categories). These envelopes can be monthly expenses like rent, utilities, car payment or they can be savings like an emergency fund. \n\nWhat ynab does better than any other budgeting app I\'ve tried is that it makes you accountable. While the app does have auto import from accounts,  it encourages you to manually enter transactions as you make them. This makes you immediately "feel" the transaction as if it were cash leaving your wallet. Likewise, when you enter a transaction, it immediately subtracts that amount from your categories. That means that if you only put $30 in groceries, but you spent $50 at the store, you need to figure out where that extra $20 will come from. Let\'s say you put $20 in your "dining out" envelope, but haven\'t spent it yet. You can tell ynab to move that $20 from "dining out" to "groceries" to cover that spending. It forces you to deal with the reality of the spending right then rather than just track that you spent more than you wanted to. It\'s more hands on which helps keep you engaged in your finances rather than a passive bystander.',
  'score': 23,
  'author': 'Nolegrl',
  'replies': []}}
Output 3: [
    {
        "name": "YNAB",
        "recommendedBy": ["ke4p04e"],
        "notRecommendedBy": []
    }
]

Input 4: 
"""

client = Client()
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "system", "content": system_msg},
              {"role": "user", "content": user_msg}],
)
print(response.choices[0].message.content)